In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gensim

In [ ]:
train = pd.read_csv('ratings_train.txt', delimiter='\t')
test = pd.read_csv('ratings_test.txt', delimiter='\t')
y_train = train.label.values
y_test = test.label.values

In [ ]:
model = gensim.models.Word2Vec.load('./vectors.bin')

In [ ]:
len(model.wv.vocab)

In [ ]:
train_sents = np.load('train_sents.npy')
train_segs = np.load('train_segs.npy')
test_sents = np.load('test_sents.npy')
test_segs = np.load('test_segs.npy')

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(embedding_init, feed_dict={embedding_placeholder: embedding_vector})
total_batch = int(len(train_idx) / batch_size)

for epoch in range(n_epoch):
    cost_sum = 0
    print('< epoch :', (epoch+1), '>')
    for i in range(total_batch):
        if i == (total_batch-1):
            batch_xs = train_idx[(i*batch_size):len(train_idx)]
            batch_ys = y_train[(i*batch_size):len(train_idx)]
            seq_length_data = get_seq_length(train_segs[(i*batch_size):len(train_idx)], max_len)
        else:
            batch_xs = train_idx[i*batch_size:(i+1)*batch_size]
            batch_ys = y_train[i*batch_size:(i+1)*batch_size]
            seq_length_data = get_seq_length(train_segs[(i*batch_size):(i+1)*batch_size], max_len)
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, seq_len: seq_length_data})
        cost_sum += cost_val
        if i % 500 == 499:
            print('%04d' % (i+1), 'Cost: ', '{:.3f}'.format(cost_sum/500))
            cost_sum = 0

In [ ]:
test_batch = int(len(test_idx) / batch_size)
test_acc = 0

for i in range(test_batch):
    if i == (test_batch-1):
        batch_xs = test_idx[(i*batch_size):len(test_idx)]
        batch_ys = y_test[(i*batch_size):len(test_idx)]
        seq_length_data = get_seq_length(test_segs[(i*batch_size):len(test_idx)], max_len)
    else:
        batch_xs = test_idx[i*batch_size:(i+1)*batch_size]
        batch_ys = y_test[i*batch_size:(i+1)*batch_size]
        seq_length_data = get_seq_length(test_segs[(i*batch_size):(i+1)*batch_size], max_len)
    acc = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys, seq_len: seq_length_data})
    test_acc += acc
print('Accuracy: ', '{:.3f}'.format(test_acc/test_batch))